# Tensorflow Object Detection API and AWS Sagemaker

In [1]:
#ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.config

In this notebook, you will train and evaluate different models using the [Tensorflow Object Detection API](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/) and [AWS Sagemaker](https://aws.amazon.com/sagemaker/). 

If you ever feel stuck, you can refer to this [tutorial](https://aws.amazon.com/blogs/machine-learning/training-and-deploying-models-using-tensorflow-2-with-the-object-detection-api-on-amazon-sagemaker/).

## Dataset

We are using the [Waymo Open Dataset](https://waymo.com/open/) for this project. The dataset has already been exported using the tfrecords format. The files have been created following the format described [here](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html#create-tensorflow-records). You can find data stored on [AWS S3](https://aws.amazon.com/s3/), AWS Object Storage. The images are saved with a resolution of 640x640.

In [2]:
%%capture
%pip install tensorflow_io sagemaker -U

In [3]:
import os
import sagemaker
from sagemaker.estimator import Estimator
from framework import CustomFramework

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


Save the IAM role in a variable called `role`. This would be useful when training the model.

In [4]:
role = sagemaker.get_execution_role()
print(role)

arn:aws:iam::359346771395:role/service-role/AmazonSageMaker-ExecutionRole-20240516T125452


In [5]:
# The train and val paths below are public S3 buckets created by Udacity for this project
inputs = {'train': 's3://cd2688-object-detection-tf2/train/', 
          'val': 's3://cd2688-object-detection-tf2/val/'} 

# Insert path of a folder in your personal S3 bucket to store tensorboard logs.
tensorboard_s3_prefix = 's3://udacity-selfdriving-240516-1358/logs/'

## Container

To train the model, you will first need to build a [docker](https://www.docker.com/) container with all the dependencies required by the TF Object Detection API. The code below does the following:
* clone the Tensorflow models repository
* get the exporter and training scripts from the repository
* build the docker image and push it 
* print the container name

In [6]:
%%bash

# clone the repo and get the scripts
# git clone https://github.com/tensorflow/models.git docker/models

# get model_main and exporter_main files from TF2 Object Detection GitHub repository
# cp docker/models/research/object_detection/exporter_main_v2.py source_dir 
# cp docker/models/research/object_detection/model_main_tf2.py source_dir

In [7]:
# build and push the docker image. This code can be commented out after being run once.
# This will take around 10 mins.

#image_name = 'tf2-object-detection'
#!sh ./docker/build_and_push.sh $image_name

To verify that the image was correctly pushed to the [Elastic Container Registry](https://aws.amazon.com/ecr/), you can look at it in the AWS webapp. For example, below you can see that three different images have been pushed to ECR. You should only see one, called `tf2-object-detection`.
![ECR Example](../data/example_ecr.png)


In [8]:
# display the container name
with open (os.path.join('docker', 'ecr_image_fullname.txt'), 'r') as f:
    container = f.readlines()[0][:-1]

print(container)

359346771395.dkr.ecr.us-east-1.amazonaws.com/tf2-object-detection:20240527120237


## Pre-trained model from model zoo

As often, we are not training from scratch and we will be using a pretrained model from the TF Object Detection model zoo. You can find pretrained checkpoints [here](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md). Because your time is limited for this project, we recommend to only experiment with the following models:
* SSD MobileNet V2 FPNLite 640x640	
* SSD ResNet50 V1 FPN 640x640 (RetinaNet50)	
* Faster R-CNN ResNet50 V1 640x640	
* EfficientDet D1 640x640	
* Faster R-CNN ResNet152 V1 640x640	

In the code below, the EfficientDet D1 model is downloaded and extracted. This code should be adjusted if you were to experiment with other architectures.

In [9]:
%%bash
pwd
rm -r /tmp/checkpoint
rm -r source_dir/checkpoint
mkdir /tmp/checkpoint
mkdir source_dir/checkpoint

#wget -O /tmp/efficientdet.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d1_coco17_tpu-32.tar.gz
#tar -zxvf /tmp/efficientdet.tar.gz --strip-components 2 --directory source_dir/checkpoint efficientdet_d1_coco17_tpu-32/checkpoint

#ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8
wget -O /tmp/ssdmobilenet.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz
tar -zxvf /tmp/ssdmobilenet.tar.gz --strip-components 2 --directory source_dir/checkpoint ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint
# result out of memory with pipeline_ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.config

# Faster R-CNN ResNet152 V1 640x640
#wget -O /tmp/fasterrcnn.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet152_v1_640x640_coco17_tpu-8.tar.gz
#tar -zxvf /tmp/fasterrcnn.tar.gz --strip-components 2 --directory source_dir/checkpoint faster_rcnn_resnet152_v1_640x640_coco17_tpu-8/checkpoint

#SSD ResNet50 V1 FPN 640x640 (RetinaNet50)
#wget -O /tmp/ssdresnet50.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
#tar -zxvf /tmp/ssdresnet50.tar.gz --strip-components 2 --directory source_dir/checkpoint ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint

# Faster R-CNN ResNet50 V1 640x640
#wget -O /tmp/fasterrcnn50.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz
#tar -zxvf /tmp/fasterrcnn50.tar.gz --strip-components 2 --directory source_dir/checkpoint faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/checkpoint


/home/ec2-user/SageMaker/cd2688-object-detection-in-urban-environment-project/1_model_training


--2024-06-06 12:58:21--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.253.122.207, 172.253.63.207, 142.250.31.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|172.253.122.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20518283 (20M) [application/x-tar]
Saving to: ‘/tmp/ssdmobilenet.tar.gz’

     0K .......... .......... .......... .......... ..........  0% 15.3M 1s
    50K .......... .......... .......... .......... ..........  0% 27.9M 1s
   100K .......... .......... .......... .......... ..........  0% 30.3M 1s
   150K .......... .......... .......... .......... ..........  0% 25.1M 1s
   200K .......... .......... .......... .......... ..........  1% 26.6M 1s
   250K .......... .......... .......... .......... ..........  1% 25.0M 1s
   300K .......... .......... .......... .......... ...

  5100K .......... .......... .......... .......... .......... 25% 60.8M 0s
  5150K .......... .......... .......... .......... .......... 25%  246M 0s
  5200K .......... .......... .......... .......... .......... 26% 63.2M 0s
  5250K .......... .......... .......... .......... .......... 26% 48.8M 0s
  5300K .......... .......... .......... .......... .......... 26% 72.2M 0s
  5350K .......... .......... .......... .......... .......... 26% 55.4M 0s
  5400K .......... .......... .......... .......... .......... 27%  327M 0s
  5450K .......... .......... .......... .......... .......... 27% 54.8M 0s
  5500K .......... .......... .......... .......... .......... 27% 61.4M 0s
  5550K .......... .......... .......... .......... .......... 27% 55.4M 0s
  5600K .......... .......... .......... .......... .......... 28% 66.0M 0s
  5650K .......... .......... .......... .......... .......... 28%  303M 0s
  5700K .......... .......... .......... .......... .......... 28% 56.8M 0s
  5750K ....

 10500K .......... .......... .......... .......... .......... 52% 86.4M 0s
 10550K .......... .......... .......... .......... .......... 52% 92.0M 0s
 10600K .......... .......... .......... .......... .......... 53% 69.0M 0s
 10650K .......... .......... .......... .......... .......... 53% 60.9M 0s
 10700K .......... .......... .......... .......... .......... 53% 66.7M 0s
 10750K .......... .......... .......... .......... .......... 53%  289M 0s
 10800K .......... .......... .......... .......... .......... 54% 57.6M 0s
 10850K .......... .......... .......... .......... .......... 54% 57.8M 0s
 10900K .......... .......... .......... .......... .......... 54% 54.5M 0s
 10950K .......... .......... .......... .......... .......... 54% 66.4M 0s
 11000K .......... .......... .......... .......... .......... 55%  276M 0s
 11050K .......... .......... .......... .......... .......... 55% 54.6M 0s
 11100K .......... .......... .......... .......... .......... 55% 60.2M 0s
 11150K ....

 15900K .......... .......... .......... .......... .......... 79%  336M 0s
 15950K .......... .......... .......... .......... .......... 79%  242M 0s
 16000K .......... .......... .......... .......... .......... 80%  242M 0s
 16050K .......... .......... .......... .......... .......... 80%  249M 0s
 16100K .......... .......... .......... .......... .......... 80%  168M 0s
 16150K .......... .......... .......... .......... .......... 80%  214M 0s
 16200K .......... .......... .......... .......... .......... 81%  156M 0s
 16250K .......... .......... .......... .......... .......... 81%  228M 0s
 16300K .......... .......... .......... .......... .......... 81%  228M 0s
 16350K .......... .......... .......... .......... .......... 81%  212M 0s
 16400K .......... .......... .......... .......... .......... 82%  200M 0s
 16450K .......... .......... .......... .......... .......... 82%  266M 0s
 16500K .......... .......... .......... .......... .......... 82%  146M 0s
 16550K ....

ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint/ckpt-0.index


## Edit pipeline.config file

The [`pipeline.config`](source_dir/pipeline.config) in the `source_dir` folder should be updated when you experiment with different models. The different config files are available [here](https://github.com/tensorflow/models/tree/master/research/object_detection/configs/tf2).

>Note: The provided `pipeline.config` file works well with the `EfficientDet` model. You would need to modify it when working with other models.

## Launch Training Job

Now that we have a dataset, a docker image and some pretrained model weights, we can launch the training job. To do so, we create a [Sagemaker Framework](https://sagemaker.readthedocs.io/en/stable/frameworks/index.html), where we indicate the container name, name of the config file, number of training steps etc.

The `run_training.sh` script does the following:
* train the model for `num_train_steps` 
* evaluate over the val dataset
* export the model

Different metrics will be displayed during the evaluation phase, including the mean average precision. These metrics can be used to quantify your model performances and compare over the different iterations.

You can also monitor the training progress by navigating to **Training -> Training Jobs** from the Amazon Sagemaker dashboard in the Web UI.

In [10]:
tensorboard_output_config = sagemaker.debugger.TensorBoardOutputConfig(
    s3_output_path=tensorboard_s3_prefix,
    container_local_output_path='/opt/training/'
)

estimator = CustomFramework(
    role=role,
    image_uri=container,
    entry_point='run_training.sh',
    source_dir='source_dir/',
    hyperparameters={
        "model_dir": "/opt/training",        
        "pipeline_config_path": "ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.config",
        "num_train_steps": "2000",    
        "sample_1_of_n_eval_examples": "1"
    },
    instance_count=1,
    instance_type='ml.g5.xlarge', # ml.g5.xlarge
    tensorboard_output_config=tensorboard_output_config,
    disable_profiler=True,
    base_job_name='tf2-object-detection'
)

estimator.fit(inputs)

INFO:sagemaker:Creating training-job with name: tf2-object-detection-2024-06-06-12-58-22-523


2024-06-06 12:58:23 Starting - Starting the training job......
2024-06-06 12:59:23 Downloading - Downloading input data...
2024-06-06 12:59:43 Downloading - Downloading the training image.........
2024-06-06 13:01:14 Training - Training image download completed. Training in progress...2024-06-06 13:01:26,846 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-06-06 13:01:26,881 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-06-06 13:01:26,915 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-06-06 13:01:26,930 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "train": "/opt/ml/input/data/train",
        "val": "/opt/ml/input/data/val"
    },
    "current_host": "algo-1",
    "current_instance_group": "homogeneousCluster",
    "current_instance_group_hosts

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
W0606 13:01:39.820333 140446448863040 deprecation.py:364] From /usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1176: sparse_to_dense (from tensorflow.python.ops.sparse_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
Instructions for updating:
`seed2` arg is deprecated.Use sample_distorted_bounding_box_v2 instead.
W0606 13:01:42.530819 140446448863040 deprecation.py:364] From /usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1176: sample_distorted_bounding_box (from tensorflow.python.ops.image_ops_impl) is deprecated and will be removed in a future version.
Instructions for updating:
`seed2` arg is deprecated.Use sample_distorted_bounding_box_v2 instead.
Instructions for updating:
Use `tf.cast` instead.
W0606 13:01:43.913228 140446

INFO:tensorflow:Step 300 per-step time 0.262s
I0606 13:03:56.938635 140446448863040 model_lib_v2.py:705] Step 300 per-step time 0.262s
INFO:tensorflow:{'Loss/classification_loss': 0.2932214,
 'Loss/localization_loss': 0.38140234,
 'Loss/regularization_loss': 0.15133281,
 'Loss/total_loss': 0.8259565,
 'learning_rate': 0.0426662}
I0606 13:03:56.938876 140446448863040 model_lib_v2.py:708] {'Loss/classification_loss': 0.2932214,
 'Loss/localization_loss': 0.38140234,
 'Loss/regularization_loss': 0.15133281,
 'Loss/total_loss': 0.8259565,
 'learning_rate': 0.0426662}
INFO:tensorflow:Step 400 per-step time 0.262s
I0606 13:04:23.115532 140446448863040 model_lib_v2.py:705] Step 400 per-step time 0.262s
INFO:tensorflow:{'Loss/classification_loss': 0.2803574,
 'Loss/localization_loss': 0.2869433,
 'Loss/regularization_loss': 0.15124226,
 'Loss/total_loss': 0.71854293,
 'learning_rate': 0.047999598}
I0606 13:04:23.115782 140446448863040 model_lib_v2.py:708] {'Loss/classification_loss': 0.2803574

INFO:tensorflow:Step 1700 per-step time 0.262s
I0606 13:10:04.555896 140446448863040 model_lib_v2.py:705] Step 1700 per-step time 0.262s
INFO:tensorflow:{'Loss/classification_loss': 0.18279874,
 'Loss/localization_loss': 0.25754672,
 'Loss/regularization_loss': 0.14939754,
 'Loss/total_loss': 0.589743,
 'learning_rate': 0.07995972}
I0606 13:10:04.556144 140446448863040 model_lib_v2.py:708] {'Loss/classification_loss': 0.18279874,
 'Loss/localization_loss': 0.25754672,
 'Loss/regularization_loss': 0.14939754,
 'Loss/total_loss': 0.589743,
 'learning_rate': 0.07995972}
INFO:tensorflow:Step 1800 per-step time 0.262s
I0606 13:10:30.728717 140446448863040 model_lib_v2.py:705] Step 1800 per-step time 0.262s
INFO:tensorflow:{'Loss/classification_loss': 0.20592345,
 'Loss/localization_loss': 0.25356287,
 'Loss/regularization_loss': 0.14908311,
 'Loss/total_loss': 0.6085694,
 'learning_rate': 0.0799474}
I0606 13:10:30.728954 140446448863040 model_lib_v2.py:708] {'Loss/classification_loss': 0.20

I0606 13:12:02.063753 139811277080384 api.py:460] feature_map_spatial_dims: [(80, 80), (40, 40), (20, 20), (10, 10), (5, 5)]
Instructions for updating:
Use `tf.cast` instead.
W0606 13:12:16.124280 139811277080384 deprecation.py:364] From /usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1176: to_int64 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
INFO:tensorflow:Finished eval step 0
I0606 13:12:16.220850 139811277080384 model_lib_v2.py:966] Finished eval step 0
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well a

I0606 13:17:11.610541 140027842676544 api.py:460] feature_map_spatial_dims: [(80, 80), (40, 40), (20, 20), (10, 10), (5, 5)]
I0606 13:17:23.373997 140027842676544 signature_serialization.py:148] Function `call_func` contains input name(s) resource with unsupported characters which will be renamed to weightsharedconvolutionalboxpredictor_predictiontower_conv2d_3_batchnorm_feature_4_fusedbatchnormv3_readvariableop_1_resource in the SavedModel.
I0606 13:17:24.642040 140027842676544 api.py:460] feature_map_spatial_dims: [(80, 80), (40, 40), (20, 20), (10, 10), (5, 5)]
W0606 13:17:35.602106 140027842676544 save_impl.py:66] Skipping full serialization of Keras layer <object_detection.meta_architectures.ssd_meta_arch.SSDMetaArch object at 0x7f59bc1792e0>, because it is not built.
W0606 13:17:35.852410 140027842676544 save_impl.py:66] Skipping full serialization of Keras layer <keras.src.layers.convolutional.separable_conv2d.SeparableConv2D object at 0x7f59900c0880>, because it is not built.
W

I0606 13:17:45.365886 140027842676544 save.py:274] Found untraced functions such as WeightSharedConvolutionalBoxPredictor_layer_call_fn, WeightSharedConvolutionalBoxPredictor_layer_call_and_return_conditional_losses, WeightSharedConvolutionalBoxHead_layer_call_fn, WeightSharedConvolutionalBoxHead_layer_call_and_return_conditional_losses, WeightSharedConvolutionalClassHead_layer_call_fn while saving (showing 5 of 173). These functions will not be directly callable after loading.
INFO:tensorflow:Assets written to: /tmp/exported/saved_model/assets
I0606 13:17:51.642750 140027842676544 builder_impl.py:804] Assets written to: /tmp/exported/saved_model/assets
I0606 13:17:52.156851 140027842676544 fingerprinting_utils.py:48] Writing fingerprint to /tmp/exported/saved_model/fingerprint.pb
INFO:tensorflow:Writing pipeline config file to /tmp/exported/pipeline.config
I0606 13:17:52.766544 140027842676544 config_util.py:253] Writing pipeline config file to /tmp/exported/pipeline.config
2024-06-06

In [11]:
# DEBUG / TROUBLESHOOT
#ls /tmp/checkpoint
#mkdir source_dir/checkpoint

import os

# Get the current working directory
current_directory = os.getcwd()
print(current_directory)

# Extract the directory name
directory_name = os.path.basename(current_directory)

# Print the directory name
print(directory_name)

files = [f for f in os.listdir('/home/ec2-user/SageMaker/cd2688-object-detection-in-urban-environment-project/1_model_training/source_dir/checkpoint')]
for f in files:
    print(f)
    # do something

print("\n/opt")
files = [f for f in os.listdir('/opt/')]
for f in files:
    print(f)
    # do something

    
# /home/ec2-user/SageMaker/cd2688-object-detection-in-urban-environment-project/1_model_training/source_dir

/home/ec2-user/SageMaker/cd2688-object-detection-in-urban-environment-project/1_model_training
1_model_training
ckpt-0.data-00000-of-00001
checkpoint
ckpt-0.index

/opt
aws
rh
containerd
amazon
aws-ofi-nccl
install-rootless-docker.sh
lib
brazil-npm-registry
docker.tgz
rootless.tgz
.sagemaker
ml


You should be able to see your model training in the AWS webapp as shown below:
![ECR Example](../data/example_trainings.png)


## Improve on the initial model

Most likely, this initial experiment did not yield optimal results. However, you can make multiple changes to the `pipeline.config` file to improve this model. One obvious change consists in improving the data augmentation strategy. The [`preprocessor.proto`](https://github.com/tensorflow/models/blob/master/research/object_detection/protos/preprocessor.proto) file contains the different data augmentation method available in the Tf Object Detection API. Justify your choices of augmentations in the write-up.

Keep in mind that the following are also available:
* experiment with the optimizer: type of optimizer, learning rate, scheduler etc
* experiment with the architecture. The Tf Object Detection API model zoo offers many architectures. Keep in mind that the pipeline.config file is unique for each architecture and you will have to edit it.
* visualize results on the test frames using the `2_deploy_model` notebook available in this repository.

In the cell below, write down all the different approaches you have experimented with, why you have chosen them and what you would have done if you had more time and resources. Justify your choices using the tensorboard visualizations (take screenshots and insert them in your write-up), the metrics on the evaluation set and the generated animation you have created with [this tool](../2_run_inference/2_deploy_model.ipynb).

In [12]:
# your write-up goes here.